In [ ]:
from collections import defaultdict

import requests

from Bio import ExPASy, SwissProt

In [ ]:
#explain why not biopython
server = 'http://www.uniprot.org/uniprot'
def do_request(server, ID='', **kwargs):
    params = ''
    req = requests.get('%s/%s%s' % (server, ID, params),params=kwargs)
    if not req.ok:
        req.raise_for_status()
    return req

In [ ]:
req = do_request(server, query='gene:p53 AND reviewed:yes',# AND organism:Human',
                 format='tab',
                 columns='id,entry name,length,organism,organism-id,database(PDB),database(HGNC)',
                 limit='50')
#We might revisit this for KEGG

In [ ]:
#XXX - stringio
import pandas as pd
import io

uniprot_list = pd.read_table(io.StringIO(req.text))
uniprot_list.rename(columns={'Organism ID': 'ID'}, inplace=True)
uniprot_list

In [ ]:
p53_human = uniprot_list[uniprot_list.ID == 9606]['Entry'].tolist()[0]

In [ ]:
handle = ExPASy.get_sprot_raw(p53_human)

In [ ]:
sp_rec= SwissProt.read(handle)

In [ ]:
print(sp_rec.entry_name, sp_rec.sequence_length, sp_rec.gene_name)
print(sp_rec.description)
print(sp_rec.organism, sp_rec.seqinfo)
print(sp_rec.sequence)

In [ ]:
print(sp_rec.comments)
print(sp_rec.keywords)

In [ ]:
help(sp_rec)

In [ ]:
done_features = set()
print(len(sp_rec.features))
for feature in sp_rec.features:
    if feature[0] in done_features:
        continue
    else:
        done_features.add(feature[0])
        print(feature)
print(len(sp_rec.cross_references))
per_source = defaultdict(list)
for xref in sp_rec.cross_references:
    source = xref[0]
    per_source[source].append(xref[1:])
print(per_source.keys())
done_GOs = set()
print(len(per_source['GO']))
for annot in per_source['GO']:
    if annot[1][0] in done_GOs:
        continue
    else:
        done_GOs.add(annot[1][0])
        print(annot)